In [1570]:
import random

In [1571]:
base_pairs = {'A': 'T', 'T': 'A', 'C': 'G', 'G': 'C'}

In [1572]:
strand_length = 10

In [1573]:
matrices = [
    # Matrix 0:
    [
        [0, 1], # a
        [1, 0]  # b
       # c  d
    ],
    # Matrix 1:
    [
        [1, 0], # c
        [0, 1]  # d
       # e  f
    ],
    # Matrix 2:
    [
        [0, 1], # e
        [1, 0]  # f
       # g  h
    ],
    # Matrix 3:
    [
        [1, 0], # g
        [0, 1]  # h
       # i  j
    ]
]

In [1574]:
vertex_labels = {
    (0, 0): 'a',
    (0, 1): 'b',
    (1, 0): 'c',
    (1, 1): 'd',
    (2, 0): 'e',
    (2, 1): 'f',
    (3, 0): 'g',
    (3, 1): 'h',
    (4, 0): 'i',
    (4, 1): 'j'
}

In [1575]:
assert strand_length % 2 == 0

matrix_count = len(matrices)
assert matrix_count > 1

assert all(len(matrices[i]) > 0 for i in range(matrix_count))
for i in range(matrix_count):
    assert all(len(matrices[i][j]) > 0 for j in range(len(matrices[i])))
    for j in range(len(matrices[i])):
        assert all(not isinstance(matrices[i][j][k], list) for k in range(len(matrices[i][j])))
        assert all((matrices[i][j][k] in (0, 1)) for k in range(len(matrices[i][j])))
assert all(len(set(len(matrices[i][j]) for j in range(len(matrices[i])))) == 1 for i in range(matrix_count))
assert all(len(matrices[i - 1][0]) == len(matrices[i]) for i in range(1, matrix_count))

vertex_count = len(matrices[0]) + sum(len(matrices[i]) for i in range(1, matrix_count)) + len(matrices[matrix_count - 1][0])
initial_vertices, terminal_vertices = len(matrices[0][0]), len(matrices[-1][0])

assert len(vertex_labels) == vertex_count
assert all(isinstance(key, tuple) and len(key) == 2 and isinstance(key[0], int) and isinstance(key[1], int) for key in vertex_labels.keys())
for i in range(matrix_count + 1):
    assert all((i, j) in vertex_labels.keys() for j in range(len(matrices[i]) if i < matrix_count else terminal_vertices))
assert all(isinstance(value, str) and len(value) > 0 for value in vertex_labels.values())

In [1576]:
def GenerateRandomDNASequence(sequence_length):
    return ''.join(random.choices(list(base_pairs.keys()), k = sequence_length))

In [1577]:
vertex_strands = []
vertex_encodings = {}
vertex_decodings = {}

for i in range(matrix_count + 1):
    vertex_layer = []
    for j in range(len(matrices[i]) if i < matrix_count else terminal_vertices):
        new_strand = GenerateRandomDNASequence(strand_length)
        
        vertex_layer.append(new_strand)
        vertex_encodings[vertex_labels[(i, j)]] = new_strand
        vertex_decodings[new_strand] = (i, j)
    vertex_strands.append(vertex_layer)

print("Vertex Strands:")
for vertex_label, vertex_strand in vertex_encodings.items():
    print("\t" + vertex_label + " : " + vertex_strand)

Vertex Strands:
	a : ATTCGTTCTT
	b : TCTTGGACAG
	c : GCACAGTTAA
	d : ATCACACAAT
	e : GCCCTTCTGG
	f : TAAAACGGCT
	g : GCTTGGACAG
	h : CTCAGTGAGC
	i : GTCATGAGCG
	j : ATGAACGGTG


In [1578]:
def GenerateComplementaryDNASequence(sequence):
    return ''.join(base_pairs[sequence[i]] for i in range(len(sequence)))

In [1579]:
edge_strands = []
edge_encodings = {}
edge_decodings = {}

for i in range(matrix_count):
    for j in range(len(matrices[i])):
        for k in range(len(matrices[i][j])):
            if matrices[i][j][k] == 1:
                left_half = GenerateComplementaryDNASequence(vertex_strands[i][j][strand_length // 2:])
                right_half = GenerateComplementaryDNASequence(vertex_strands[i + 1][k][:strand_length // 2])
                new_strand = left_half + right_half

                edge_strands.append(new_strand)
                edge_encodings[(vertex_labels[(i, j)], vertex_labels[(i + 1, k)])] = new_strand
                edge_decodings[new_strand] = (vertex_labels[(i, j)], vertex_labels[(i + 1, k)])

print("Edge Strands:")
for edge_strand, edge_label in edge_decodings.items():
    print("\t(" + edge_label[0] + "," + edge_label[1] + ") : " + edge_strand)

Edge Strands:
	(a,d) : AAGAATAGTG
	(b,c) : CTGTCCGTGT
	(c,e) : CAATTCGGGA
	(d,f) : TGTTAATTTT
	(e,h) : AGACCGAGTC
	(f,g) : GCCGACGAAC
	(g,i) : CTGTCCAGTA
	(h,j) : ACTCGTACTT


In [1580]:
template = set()
for i in range(matrix_count + 1):
    for j in range(len(matrices[i]) if i < matrix_count else terminal_vertices):
        template.add(vertex_strands[i][j])

In [1581]:
def Match(strandOne, strandTwo):
    strand_len = len(strandOne)
    return strand_len == len(strandTwo) and all(strandOne[i] == base_pairs[strandTwo[i]] for i in range(strand_len))

In [1582]:
for _ in range(matrix_count):
    growing_strands = set()
    for path_strand in template:
        for edge_strand in edge_strands:
            if Match(path_strand[-(strand_length // 2):], edge_strand[:strand_length // 2]):
                growing_strands.add(path_strand + "-" + vertex_encodings[edge_decodings[edge_strand][1]])
    template = template.union(growing_strands)

print("Path Strands:")
for path_strand in template:
    print("\t" + path_strand + " (" + vertex_labels[vertex_decodings[path_strand[:strand_length]]], end = '')
    for i in range(1, (len(path_strand) + 1) // (strand_length + 1)):
        print(" -> " + vertex_labels[vertex_decodings[path_strand[i * (strand_length + 1) : (i + 1) * (strand_length + 1) - 1]]], end = '')
    print(")")

Path Strands:
	GCTTGGACAG-GTCATGAGCG (g -> i)
	GCCCTTCTGG (e)
	TCTTGGACAG-GCACAGTTAA-GCCCTTCTGG (b -> c -> e)
	TAAAACGGCT (f)
	TAAAACGGCT-GCTTGGACAG-GCACAGTTAA-GCCCTTCTGG (f -> g -> c -> e)
	GCCCTTCTGG-CTCAGTGAGC (e -> h)
	CTCAGTGAGC-ATGAACGGTG (h -> j)
	GCACAGTTAA-GCCCTTCTGG-CTCAGTGAGC-ATGAACGGTG (c -> e -> h -> j)
	ATTCGTTCTT-ATCACACAAT-TAAAACGGCT-GCTTGGACAG-GTCATGAGCG (a -> d -> f -> g -> i)
	GCTTGGACAG (g)
	TCTTGGACAG-GCACAGTTAA (b -> c)
	ATTCGTTCTT-ATCACACAAT-TAAAACGGCT (a -> d -> f)
	GCTTGGACAG-GCACAGTTAA (g -> c)
	GCACAGTTAA (c)
	ATTCGTTCTT-ATCACACAAT-TAAAACGGCT-GCTTGGACAG-GCACAGTTAA (a -> d -> f -> g -> c)
	TCTTGGACAG-GCACAGTTAA-GCCCTTCTGG-CTCAGTGAGC-ATGAACGGTG (b -> c -> e -> h -> j)
	TCTTGGACAG (b)
	ATTCGTTCTT (a)
	ATCACACAAT-TAAAACGGCT-GCTTGGACAG-GCACAGTTAA-GCCCTTCTGG (d -> f -> g -> c -> e)
	TAAAACGGCT-GCTTGGACAG-GCACAGTTAA-GCCCTTCTGG-CTCAGTGAGC (f -> g -> c -> e -> h)
	TCTTGGACAG-GTCATGAGCG (b -> i)
	GCTTGGACAG-GCACAGTTAA-GCCCTTCTGG (g -> c -> e)
	TAAAACGGCT-GCTTGGACAG (f 

In [1583]:
print("Complete Paths:")

product_matrix = [[0 for j in range(terminal_vertices)] for i in range(initial_vertices)]

for path_strand in template:
    starting_vertex = path_strand[:strand_length]
    ending_vertex = path_strand[-strand_length:]

    if starting_vertex in vertex_strands[0] and ending_vertex in vertex_strands[matrix_count]:
        product_matrix[vertex_decodings[starting_vertex][1]][vertex_decodings[ending_vertex][1]] = 1
        
        print("\t" + path_strand + " (" + vertex_labels[vertex_decodings[path_strand[:strand_length]]], end = '')
        for i in range(1, (len(path_strand) + 1) // (strand_length + 1)):
            print(" -> " + vertex_labels[vertex_decodings[path_strand[i * (strand_length + 1) : (i + 1) * (strand_length + 1) - 1]]], end = '')
        print(")")

print("\nProduct Matrix:")
print("\t", product_matrix)

Complete Paths:
	ATTCGTTCTT-ATCACACAAT-TAAAACGGCT-GCTTGGACAG-GTCATGAGCG (a -> d -> f -> g -> i)
	TCTTGGACAG-GCACAGTTAA-GCCCTTCTGG-CTCAGTGAGC-ATGAACGGTG (b -> c -> e -> h -> j)
	TCTTGGACAG-GTCATGAGCG (b -> i)

Product Matrix:
	 [[1, 0], [1, 1]]
